In [ ]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np

df_distances = pd.read_csv(r'.\\FULLDATASET.csv')

even = np.arange(0,23,2)
odd = np.arange(1,23,2)

X_train = df_distances.loc[odd, 'Distance (1 to 2)':'Distance (67 to 68)']
y_train = df_distances.loc[odd,'Result']

X_test = df_distances.loc[even, 'Distance (1 to 2)':'Distance (67 to 68)']
y_test = df_distances.loc[even,'Result']

cl = MLPClassifier(random_state=50, max_iter=10000).fit(X_train, y_train)

print(cl.predict(X_test))
print('\n',y_test.T)
cl.score(X_test,y_test)

['W' 'L' 'W' 'W' 'L' 'W' 'L' 'W' 'W' 'W' 'L' 'L']

 0     W
2     W
4     W
6     W
8     W
10    W
12    L
14    L
16    L
18    L
20    L
22    L
Name: Result, dtype: object


0.5833333333333334